In [17]:
from transformers import BertModel,BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
#from pytorch_pretrained_bert import BertModel
model=BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True,)
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [2]:
text="sample text."
text="[CLS] "+text+" [SEP]"
text=text+"the other sentence."
text=text+"third"
tokens=tokenizer.tokenize(text)
seg_ids=[1]*len(tokens)
print(tokens)
print(seg_ids)


['[CLS]', 'sample', 'text', '.', '[SEP]', 'the', 'other', 'sentence', '.', 'third']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [3]:
ids=tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[101, 7099, 3793, 1012, 102, 1996, 2060, 6251, 1012, 2353]


In [4]:
import torch
ids_t=torch.tensor(ids)
seg_t=torch.tensor(seg_ids)
print(ids_t)
ids_t=torch.tensor(ids_t).unsqueeze(0)
print(ids_t)

tensor([ 101, 7099, 3793, 1012,  102, 1996, 2060, 6251, 1012, 2353])
tensor([[ 101, 7099, 3793, 1012,  102, 1996, 2060, 6251, 1012, 2353]])


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10572\3788750668.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ids_t=torch.tensor(ids_t).unsqueeze(0)


In [5]:
with torch.no_grad():
    outputs=model(ids_t)
    embed=(outputs[0])
    print(len(outputs))
    
#print(embed.shape)
    h0=outputs[0]
    h1=outputs[1]
    h2=outputs[2]
    print("shapes ",h0.shape,"  ",h1.shape,"  ")
    print(len(h0),"  ",len(h1),"  ",len(h2))
    #print(len(h0[0]))
    #print(len(h1[0]))
    print(h0[0].shape)
    print(h1[0].shape)
    print(h0[0][0].shape)
    print(len(h2[0]))
    print(len(h2[0][0]))
    print(len(h2[0][0][0]))

    print(h0[0])  #=h1[0]
    print(h0[0][0])
    print(h0[0][3][767])


3
shapes  torch.Size([1, 10, 768])    torch.Size([1, 768])   
1    1    13
torch.Size([10, 768])
torch.Size([768])
torch.Size([768])
1
10
768
tensor([[-0.4880, -0.1663, -0.2095,  ..., -0.3476,  0.4867,  0.4252],
        [ 0.2041, -0.3386, -0.1902,  ..., -0.1832,  0.5613, -0.0529],
        [ 0.3351, -0.0933,  0.1660,  ..., -0.4334,  0.2555,  0.3301],
        ...,
        [-0.0088, -0.5130, -0.0548,  ..., -0.0487,  0.0143, -0.1981],
        [-0.7077, -0.6725,  0.3904,  ...,  0.4060,  0.4872, -0.1999],
        [-0.2843, -0.5059,  0.4111,  ..., -0.3996,  0.7107, -0.6139]])
tensor([-4.8800e-01, -1.6625e-01, -2.0947e-01, -2.1812e-01, -3.0280e-01,
         2.6438e-02,  2.4399e-01,  4.5131e-01, -3.2714e-01, -2.5757e-01,
        -4.9264e-01, -7.0161e-02, -1.4464e-01,  1.3770e-01,  2.3711e-01,
         3.0323e-01,  1.2400e-03,  2.9459e-01,  2.6913e-01, -5.6465e-01,
        -6.4518e-02,  3.6456e-02, -1.7703e-01, -2.1185e-01,  1.8423e-01,
        -2.3581e-01, -6.2788e-02, -1.7443e-01, -5.3198e-02,

In [6]:
print(len(h2))
embedings=torch.stack(h2,dim=0)
print(len(embedings[0]))
print(len(embedings[0][0]))
print(len(embedings[0][0][0]))
print(embedings.shape)
embedings=torch.squeeze(embedings,dim=1)
print(embedings.shape)
embedings=embedings.permute(1,0,2)
print(embedings.shape)





13
1
10
768
torch.Size([13, 1, 10, 768])
torch.Size([13, 10, 768])
torch.Size([10, 13, 768])


In [7]:
#sentiment analysis using weights of Bert and then Logistic Regression
import numpy as np
import pandas as pd 
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
 

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10572\1276239137.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [8]:
df.head()
df1=df[0:400]
df1.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [9]:
tokenized=df1[0].apply((lambda x: tokenizer.encode(x,add_special_tokens=True) ))
print(tokenized)

0      [101, 1037, 18385, 1010, 6057, 1998, 2633, 182...
1      [101, 4593, 2128, 27241, 23931, 2013, 1996, 62...
2      [101, 2027, 3653, 23545, 2037, 4378, 24185, 10...
3      [101, 2023, 2003, 1037, 17453, 14726, 19379, 1...
4      [101, 5655, 6262, 1005, 1055, 12075, 2571, 376...
                             ...                        
395    [101, 1996, 2732, 2040, 3271, 2507, 1037, 1212...
396    [101, 1037, 2204, 2143, 2007, 1037, 5024, 2187...
397    [101, 1037, 28851, 2836, 2011, 10786, 8840, 13...
398    [101, 1037, 4030, 3048, 2610, 24508, 10874, 20...
399    [101, 2144, 4830, 14227, 2121, 16511, 2000, 31...
Name: 0, Length: 400, dtype: object


In [10]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

print(max_len)
padded=np.array([i+[0]*(max_len-len(i))for i in tokenized.values])
inp=torch.tensor(padded)
print(inp)

54
tensor([[  101,  1037, 18385,  ...,     0,     0,     0],
        [  101,  4593,  2128,  ...,     0,     0,     0],
        [  101,  2027,  3653,  ...,     0,     0,     0],
        ...,
        [  101,  1037, 28851,  ...,     0,     0,     0],
        [  101,  1037,  4030,  ...,     0,     0,     0],
        [  101,  2144,  4830,  ...,     0,     0,     0]], dtype=torch.int32)


In [11]:
attention=np.where(padded!=0,1,0)
print(attention.shape)
print(attention)
print(inp.shape)

(400, 54)
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]
torch.Size([400, 54])


In [12]:
model2=BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True,)
attention=torch.tensor(attention)
with torch.no_grad():
    last_hidden=model2(inp,attention_mask=attention)
#last=last_hidden[0]


In [13]:
features=last_hidden[0][:,0,:].numpy()
labels=df1[1]

In [14]:
train_features,test_features,train_labels,test_labels=train_test_split(features,labels)

In [15]:
LR=LogisticRegression()
LR.fit(train_features,train_labels)
LR.score(test_features,test_labels)

c:\Program Files\Git\Projects\1\1\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.82

In [16]:
import sklearn.metrics as met
predict=LR.predict(test_features)
print(met.accuracy_score(predict,test_labels))
print(met.average_precision_score(predict,test_labels))
print(met.f1_score(predict,test_labels))
cm=met.confusion_matrix(test_labels,predict)
print(cm)
print(cm.shape)
tn,fp,fn,tp=cm.ravel()
print(tn,fp,fn,tp)
#End of sentiment analsis

0.82
0.8001960784313726
0.8301886792452831
[[38 11]
 [ 7 44]]
(2, 2)
38 11 7 44
